## Brief Projet : 

Vous êtes un data analyst ,vous travaillez en bénévolat pour la médiathèque de la ville qui a mis en place un site web pour la vente de certains livres de son stock.. et elle souhaite analyser les caractéristiques sa clientèle pour mieux comprendre les tendances de leurs reservations/achats de livres. 

La médiathèque vous donne accès à leur site : http://books.toscrape.com/index.html. et souhaite que vous collectez/analysez leurs données.


**Étapes du projet** :

1. **Collecte de données** : 
* Utiliser la bibliothèque `requests` pour envoyer des requêtes HTTP au site web qui répertorie les livres. Vous récupérez le contenu HTML de la page web.

* A l'aide de la bibliothèque `Beautiful Soup`, analyser le contenu HTML du site et extraire les informations pertinentes: parcourir le code HTML, identifier les balises cibles (qui contiennent les données sur les livres, telles que `<div>` ou `<li>` ) et extraire les informations pertinentes telles que le nom du livre, la catégorie, la note moyenne des avis, le nombre de livres en stock, le prix etc.

2. **Nettoyage et préparation des données** : nettoyer les valeurs, convertir les types de données si nécessaire, gérer les valeurs manquantes, etc.

3. **Stockage des données** : 
* Proposer une modélisation de base de données SQL adaptée. 
* Créer le schéma de la base de données , les différentes tables pour stocker les données propres sur les livres.

4. **Analyse des données** : 

* Faire une analyse exploratoire des données : identification de KPIs pertinents ,création de graphiques, le calcul de statistiques descriptives, l'identification de tendances, etc => pour aider la médiathèque à mieux faire son étude de clientèle. 



### Importer les librairies

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df1=pd.read_csv("books.csv")
df1.head()

,title,rating,price,availability,category
0,A Light in the Attic,Three,51.77,In stock,Books
1,Tipping the Velvet,One,53.74,In stock,Books
2,Soumission,One,50.10,In stock,Books
3,Sharp Objects,Four,47.82,In stock,Books
4,Sapiens: A Brief History of Humankind,Five,54.23,In stock,Books


### Stockage de données 

* Proposer une modélisation cohérente et créer la BDD dans votre SGBDR

ON est typiquement sur un contxte de données structurées ave un SI qui permet de stocker les informations sur les livres, leurs catégories et les notations associées, on peut modéliser ce système avec 4 tables



1. **Table "Books"** :
   - **book_id** (Clé primaire) : Identifiant unique pour chaque livre.
   - **title** : Titre du livre.
   - **price** : Prix du livre.
   - **availability** : Disponibilité du livre (peut être stocké sous forme de booléen).
   - **rating_id** (Clé étrangère) : Référence à la notation du livre.


2. **Table "Categories"** :
   - **category_id** (Clé primaire) : Identifiant unique pour chaque catégorie.
   - **category_name** : Nom de la catégorie



3. **Table "Ratings"** :
   - **rating_id** (Clé primaire) : Identifiant unique pour chaque notation.
   - **rating_name** : Nom de la notation (par exemple, "One", "Two", "Three", "Four", "Five").
  

5. **Table "Correspondance_livre_catégorie"** :
   - **book_id** 
   - **category_id**  




```sql
Create Database Books;

Use Books;

CREATE TABLE Categories (
    category_id INT PRIMARY KEY,
    category_name VARCHAR(255)
);

CREATE TABLE Ratings (
    rating_id INT PRIMARY KEY,
    rating_name VARCHAR(10)
);

CREATE TABLE Books (
    book_id INT PRIMARY KEY,
    title VARCHAR(255),
    price DECIMAL(10, 2),
    availability BOOLEAN,
    category_id INT,
    rating_id INT,
    FOREIGN KEY (category_id) REFERENCES Categories(category_id),
    FOREIGN KEY (rating_id) REFERENCES Ratings(rating_id)
);
```



#### Initialisation des tables 

In [17]:
## MYSQL CONNECTOR

SQLAlchemy

In [11]:
### AVEC SQLITE

In [20]:
from sqlalchemy.engine import create_engine
import sqlite3
from pandas.io import sql
import subprocess

connection= sqlite3.connect('database_books') 


create_categories_table = '''
    CREATE TABLE IF NOT EXISTS Categories (
        category_id INTEGER PRIMARY KEY AUTOINCREMENT,
        category_name VARCHAR(255)
    );
    '''
create_ratings_table = '''
    CREATE TABLE IF NOT EXISTS Ratings (
        rating_id INTEGER PRIMARY KEY AUTOINCREMENT,
        rating_name VARCHAR(10)
    );
    '''

create_books_table = '''
    CREATE TABLE IF NOT EXISTS Books (
        book_id INTEGER PRIMARY KEY AUTOINCREMENT,
        title VARCHAR(255),
        price DECIMAL(10, 2),
        availability BOOLEAN,
        rating_id INT,
        FOREIGN KEY (rating_id) REFERENCES Ratings(rating_id) ON DELETE CASCADE
    );
    '''

create_link = '''
    CREATE TABLE IF NOT EXISTS Correspondance_Books_Cat (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        book_id,
        category_id,
        FOREIGN KEY (book_id) REFERENCES Books(book_id) ON DELETE CASCADE,
        FOREIGN KEY (category_id) REFERENCES Categories(category_id) ON DELETE CASCADE
    );
    '''


connection.execute(create_categories_table)
connection.execute(create_ratings_table)
connection.execute(create_books_table)
connection.execute(create_link)
connection.commit()

In [21]:
#Category Table
categories_df = df1[['category']].drop_duplicates()
categories_df.head()

,category
0,Books
1020,Travel
1031,Mystery
1083,Historical Fiction
1129,Sequential Art


In [22]:
for i, row in categories_df.iterrows():
    category_name = row['category']
    insert_category_query = '''INSERT INTO Categories (category_name) VALUES (?);'''
    connection.execute(insert_category_query, (category_name,))
    connection.commit()

In [23]:
ratings_df = df1[['rating']].drop_duplicates()
ratings_df

,rating
0,3
1,1
3,4
4,5
10,2


In [25]:
for _, row in ratings_df.iterrows():
    rating_name = int(row['rating'])
    insert_rating_query ='''INSERT INTO Ratings (rating_name) VALUES (?);'''
    connection.execute(insert_rating_query, (rating_name,))
    connection.commit()

In [26]:
cursor = connection.cursor()
cursor.execute("SELECT * FROM Ratings")
rows = cursor.fetchall()

In [27]:
rat_id=pd.DataFrame(rows, columns=["rating_id","rating_name"])
rat_id

,rating_id,rating_name
0,1,3
1,2,1
2,3,4
3,4,5
4,5,2


In [28]:
rat_id.dtypes

rating_id       int64
rating_name    object
dtype: object

In [27]:
#Table Livre

In [29]:
livre = df1[['title','price','availability','rating']].copy()
print(livre.shape)
print(livre.title.nunique())
livre = livre.drop_duplicates('title')

(2000, 4)
999


In [30]:
livre["rating"]=livre["rating"].astype(str)

In [31]:
livre2 = pd.merge(livre,rat_id, how='left', left_on="rating", right_on="rating_name")
livre2

,title,price,availability,rating,rating_id,rating_name
0,A Light in the Attic,51.77,True,3,1,3
1,Tipping the Velvet,53.74,True,1,2,1
2,Soumission,50.10,True,1,2,1
3,Sharp Objects,47.82,True,4,3,4
4,Sapiens: A Brief History of Humankind,54.23,True,5,4,5
...,...,...,...,...,...,...
994,Alice in Wonderland (Alice's Adventures in Won...,55.53,True,1,2,1
995,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,True,4,3,4
996,A Spy's Devotion (The Regency Spies of London #1),16.97,True,5,4,5
997,1st to Die (Women's Murder Club #1),53.98,True,1,2,1


In [32]:
for _, row in livre2.iterrows():
    title = row['title']
    price = float(row['price']) 
    availability = row['availability']
    rating_id = row['rating_id']
    insert_livre_query ='''INSERT INTO Books (title, price, availability, rating_id) VALUES (?,?,?,?);'''
    cursor.execute(insert_livre_query, (title,price,availability,rating_id))
    connection.commit()

In [43]:
### Table correspondace

#1. récupérer les id livres de sql
cursor = connection.cursor()
cursor.execute("SELECT book_id, title FROM Books")
rows = cursor.fetchall()
livre_ids = pd.DataFrame(rows, columns=["book_id","title"])


#2. récupérer les id categorie de sql
cursor = connection.cursor()
cursor.execute("SELECT category_id, category_name FROM Categories")
rows = cursor.fetchall()
cat_ids = pd.DataFrame(rows, columns=["category_id","category_name"])

In [49]:
df1 = df1.merge(livre_ids, how='left', on='title')
df1 = df1.merge(cat_ids, how='left', left_on='category', right_on = 'category_name')

In [50]:
df1

,title,rating,price,availability,category,book_id,category_id,category_name
0,A Light in the Attic,3,51.77,True,Books,1,1,Books
1,Tipping the Velvet,1,53.74,True,Books,2,1,Books
2,Soumission,1,50.10,True,Books,3,1,Books
3,Sharp Objects,4,47.82,True,Books,4,1,Books
4,Sapiens: A Brief History of Humankind,5,54.23,True,Books,5,1,Books
...,...,...,...,...,...,...,...,...
1995,Why the Right Went Wrong: Conservatism--From G...,4,52.65,True,Politics,220,48,Politics
1996,Equal Is Unfair: America's Misguided Fight Aga...,1,56.86,True,Politics,383,48,Politics
1997,Amid the Chaos,1,36.58,True,Cultural,213,49,Cultural
1998,Dark Notes,5,19.19,True,Erotica,201,50,Erotica


In [ ]:
#3 Insert book_id - category_id in sql

In [51]:
for _, row in df1.iterrows():
    book_id = row['book_id']
    category_id = row['category_id']
    insert_corr_query ='''INSERT INTO Correspondance_Books_Cat (book_id,category_id) VALUES (?,?);'''
    connection.execute(insert_corr_query, (book_id,category_id))
    connection.commit()

In [53]:
#4. Verification
cursor = connection.cursor()
cursor.execute("SELECT * FROM Correspondance_Books_Cat")
rows = cursor.fetchall()
pd.DataFrame(rows, columns=["id","book_id","category_id"])

,id,book_id,category_id
0,1,1,1
1,2,2,1
2,3,3,1
3,4,4,1
4,5,5,1
...,...,...,...
1995,1996,220,48
1996,1997,383,48
1997,1998,213,49
1998,1999,201,50
